Solve a **Markowitz Portfolio** problem with **JuMP**:
$$
\begin{alignat}{2}
\min_{x \geq 0} \quad & x^T \Sigma x \\
\text{subject to} \quad & r^T x \geq r_{min} \\
& e^T x = 1
\end{alignat}
$$

In [ ]:
using Distributions
function generate_data(n, num_obs)
    data = zeros(num_obs, n)

    # Linking factors
    beta = [(i-1.0)/n for i = 1:n] 

    for obs_ind = 1:num_obs
        # Common market factor, mean 3%, sd 2%, truncate at +- 3 sd
        z = rand(Normal(0.03, 0.02))
        z = max(z, 0.03 - 3*0.02)
        z = min(z, 0.03 + 3*0.02)

        for asset_ind = 1:n
            # Idiosyncratic contribution, mean 0%, sd 3%, truncated at +- 3 sd
            asset = rand(Normal(0.00, 0.03))
            asset = max(asset, 0.00 - 3*0.03)
            asset = min(asset, 0.00 + 3*0.03)
            data[obs_ind, asset_ind] = beta[asset_ind] * z + asset
        end
    end
    return 100*data
end
data = generate_data(5,10000);

In [ ]:
using JuMP, Interact, Gadfly
set_default_plot_size(4inch,4inch)
n, μ, Σ = 5, vec(mean(data,1)), cov(data)
@manipulate for r_min in 1.0:0.1:2.0
    port = Model()
    @defVar(port, 0 ≤ x[1:n] ≤ 1)
    @addConstraint(port, sum(x) == 1)
    @addConstraint(port, dot(μ,x) ≥ r_min)
    @setObjective(port, Min, 
        sum{Σ[i,j]*x[i]*x[j],i=1:n,j=1:n})
    solve(port)
    plot(x=data*getValue(x)[:], Geom.density,
         Guide.xlabel("Return"), Guide.ylabel("Density"))
end